In [1]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/курс_дата.csv', ';')

<ipython-input-2-d5118d8993e7>:1: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  df = pd.read_csv('/content/курс_дата.csv', ';')


In [ ]:
df

,"Course Absence Rate, x1","PF, x2","FIN, y"
0,17,21,abs
1,19%,7,NaN
2,21%,70,10
3,24%,83,40
4,21%,100,33
5,29%,82,55
6,5%,80,45
7,26%,82,10
8,0%,100,95
9,24%,7,NaN


In [ ]:
# prompt: Используется DataFrame df: delete %

df['Course Absence Rate, x1'] = df['Course Absence Rate, x1'].str.replace('%', '')
df['Course Absence Rate, x1'] = df['Course Absence Rate, x1'].astype(float)
df['Course Absence Rate, x1'] = df['Course Absence Rate, x1'] / 100
df.head()


,"Course Absence Rate, x1","PF, x2","FIN, y"
0,0.17,21,abs
1,0.19,7,NaN
2,0.21,70,10
3,0.24,83,40
4,0.21,100,33


In [ ]:
df['PF, x2'] = df['PF, x2'].replace('failed', 0)
df['PF, x2'] = df['PF, x2'].astype(float)
df['PF, x2'] = df['PF, x2'] / 100
# Преобразуем значения колонки 'PF, x2' в тип float
df['PF, x2'] = df['PF, x2'].astype(float)



In [ ]:
df.head(3)

,"Course Absence Rate, x1","PF, x2","FIN, y"
0,0.17,0.21,abs
1,0.19,0.07,NaN
2,0.21,0.70,10


In [ ]:
df['FIN, y'].unique()

array(['abs', nan, '10', '40', '33', '55', '45', '95', '65', '25', '28',
       '74', 'failed', '38', '15', '53', '50', '48', '37', '30', '35',
       '8', '0'], dtype=object)

In [ ]:
# Замена 'abs' на 0
df['FIN, y'] = df['FIN, y'].replace('abs', '0')

# Удаление nan значений
df.dropna(inplace=True)

# Замена 'failed' на 0
df['FIN, y'] = df['FIN, y'].replace('failed', '0')

# Преобразование столбца в числовой формат
df['FIN, y'] = pd.to_numeric(df['FIN, y'])

# Проверка уникальных значений после замен
print(df['FIN, y'].unique())


[ 0 10 40 33 55 45 95 65 25 28 74 38 15 53 50 48 37 30 35  8]


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33 entries, 0 to 35
Data columns (total 3 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Course Absence Rate, x1  33 non-null     float64
 1   PF, x2                   33 non-null     float64
 2   FIN, y                   33 non-null     int64  
dtypes: float64(2), int64(1)
memory usage: 1.0 KB


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from sklearn.model_selection import train_test_split

X = df['PF, x2']
y = df['FIN, y']
# Преобразование данных в тензоры PyTorch
X_tensor = torch.tensor(X.values, dtype=torch.float32)
y_tensor = torch.tensor(y.values, dtype=torch.float32)

# Разделение данных на тренировочный и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.3, random_state=42)

# Создание DataLoader для тренировочного и тестового наборов
train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)

test_data = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_data, batch_size=16, shuffle=False)

# Определение архитектуры нейронной сети
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(1, 64) # 1 входной узел, 64 выходных узла
        self.fc2 = nn.Linear(64, 64) # 64 входных узла, 64 выходных узла
        self.fc3 = nn.Linear(64, 1) # 64 входных узла, 1 выходной узел

    def forward(self, x):
        x = x.unsqueeze(1)  # добавляем размерность для соответствия размерности весов
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Инициализация модели, функции потерь и оптимизатора
model = Net()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Обучение модели
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets.unsqueeze(1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

# Оценка модели на тестовом наборе
model.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs, targets.unsqueeze(1))
        test_loss += loss.item() * inputs.size(0)
test_loss /= len(test_loader.dataset)
print(f"Test Loss: {test_loss:.4f}")


Epoch 1/100, Loss: 1777.2343
Epoch 2/100, Loss: 1768.6704
Epoch 3/100, Loss: 1759.5924
Epoch 4/100, Loss: 1750.7080
Epoch 5/100, Loss: 1741.6182
Epoch 6/100, Loss: 1732.8605
Epoch 7/100, Loss: 1723.1358
Epoch 8/100, Loss: 1713.7152
Epoch 9/100, Loss: 1702.6419
Epoch 10/100, Loss: 1692.2912
Epoch 11/100, Loss: 1681.6209
Epoch 12/100, Loss: 1669.1148
Epoch 13/100, Loss: 1657.3485
Epoch 14/100, Loss: 1643.5508
Epoch 15/100, Loss: 1630.6882
Epoch 16/100, Loss: 1616.7261
Epoch 17/100, Loss: 1601.8124
Epoch 18/100, Loss: 1586.0710
Epoch 19/100, Loss: 1571.0498
Epoch 20/100, Loss: 1553.9352
Epoch 21/100, Loss: 1536.4383
Epoch 22/100, Loss: 1518.1064
Epoch 23/100, Loss: 1500.1661
Epoch 24/100, Loss: 1480.8537
Epoch 25/100, Loss: 1461.5442
Epoch 26/100, Loss: 1438.8625
Epoch 27/100, Loss: 1417.5212
Epoch 28/100, Loss: 1395.5685
Epoch 29/100, Loss: 1372.4345
Epoch 30/100, Loss: 1349.4158
Epoch 31/100, Loss: 1325.0195
Epoch 32/100, Loss: 1299.5945
Epoch 33/100, Loss: 1274.4647
Epoch 34/100, Loss:

In [ ]:
# Создадим тензор PyTorch для значения PF, x2, которое мы хотим предсказать
pf_value = 0.8  # замените это на нужное значение PF, x2
pf_tensor = torch.tensor([[pf_value]], dtype=torch.float32)

# Получим предсказание от модели
model.eval()
with torch.no_grad():
    fin_prediction = model(pf_tensor)

print(f"Предсказанный FIN для значения PF, x2 = {pf_value}: {fin_prediction.item()}")


Предсказанный FIN для значения PF, x2 = 0.8: 36.23912811279297


## Теперь X = Course Absence Rate

In [ ]:
X = df['Course Absence Rate, x1']
y = df['FIN, y']
# Преобразование данных в тензоры PyTorch
X_tensor = torch.tensor(X.values, dtype=torch.float32)
y_tensor = torch.tensor(y.values, dtype=torch.float32)

# Разделение данных на тренировочный и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.3, random_state=42)

# Создание DataLoader для тренировочного и тестового наборов
train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)

test_data = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_data, batch_size=16, shuffle=False)

# Определение архитектуры нейронной сети
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(1, 64) # 1 входной узел, 64 выходных узла
        self.fc2 = nn.Linear(64, 64) # 64 входных узла, 64 выходных узла
        self.fc3 = nn.Linear(64, 1) # 64 входных узла, 1 выходной узел

    def forward(self, x):
        x = x.unsqueeze(1)  # добавляем размерность для соответствия размерности весов
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Инициализация модели, функции потерь и оптимизатора
model = Net()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Обучение модели
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets.unsqueeze(1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

# Оценка модели на тестовом наборе
model.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs, targets.unsqueeze(1))
        test_loss += loss.item() * inputs.size(0)
test_loss /= len(test_loader.dataset)
print(f"Test Loss: {test_loss:.4f}")


Epoch 1/100, Loss: 1802.4465
Epoch 2/100, Loss: 1795.8726
Epoch 3/100, Loss: 1789.6892
Epoch 4/100, Loss: 1784.6582
Epoch 5/100, Loss: 1779.5836
Epoch 6/100, Loss: 1774.6704
Epoch 7/100, Loss: 1770.2177
Epoch 8/100, Loss: 1766.2985
Epoch 9/100, Loss: 1762.3190
Epoch 10/100, Loss: 1758.1550
Epoch 11/100, Loss: 1753.8028
Epoch 12/100, Loss: 1749.5149
Epoch 13/100, Loss: 1744.6852
Epoch 14/100, Loss: 1739.7942
Epoch 15/100, Loss: 1734.8694
Epoch 16/100, Loss: 1729.0557
Epoch 17/100, Loss: 1723.5624
Epoch 18/100, Loss: 1717.5664
Epoch 19/100, Loss: 1711.3458
Epoch 20/100, Loss: 1704.7154
Epoch 21/100, Loss: 1697.9811
Epoch 22/100, Loss: 1690.3499
Epoch 23/100, Loss: 1682.8600
Epoch 24/100, Loss: 1675.0605
Epoch 25/100, Loss: 1666.1187
Epoch 26/100, Loss: 1657.7043
Epoch 27/100, Loss: 1648.6132
Epoch 28/100, Loss: 1639.6039
Epoch 29/100, Loss: 1628.9934
Epoch 30/100, Loss: 1618.4124
Epoch 31/100, Loss: 1607.5862
Epoch 32/100, Loss: 1597.2763
Epoch 33/100, Loss: 1585.5412
Epoch 34/100, Loss:

In [ ]:
# Создадим тензор PyTorch для значения PF, x2, которое мы хотим предсказать
pf_value = 0.8  # замените это на нужное значение PF, x2
pf_tensor = torch.tensor([[pf_value]], dtype=torch.float32)

# Получим предсказание от модели
model.eval()
with torch.no_grad():
    fin_prediction = model(pf_tensor)

print(f"Предсказанный FIN для значения Course Absence Rate = {pf_value}: {fin_prediction.item()}")


Предсказанный FIN для значения Course Absence Rate = 0.8: 38.746917724609375
